# Nodejs MNIST model Deployment

 * Wrap a nodejs tensorflow model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
 
## Dependencies

 * ```pip install seldon-core```
 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)
 * node (version>=8.11.0)
 * npm

## Train locally using npm commands
 This model takes in mnist images of size 28x28x1 as input and outputs an array of size 10 with prediction of each digits from 0-9

In [1]:
!make train && make clean_build

npm install
        ......] - refresh-package-json:argparse: timing action:finalize Co
> @tensorflow/tfjs-node@0.1.15 install /home/clive/work/seldon-core/fork-seldon-core/examples/models/nodejs_mnist/node_modules/@tensorflow/tfjs-node
> node scripts/install.js

* Downloading libtensorflow
[                              ] 188667/bps 0% 104.7s[                              ] 1942639/bps 0% 10.1s[                              ] 3945442/bps 1% 5.0s[=                             ] 5148897/bps 2% 3.8s[=                             ] 5778143/bps 2% 3.3s[=                             ] 6450370/bps 3% 3.0s[=                             ] 7358431/bps 4% 2.6s[=                             ] 7337500/bps 5% 2.6s[==                            ] 8626919/bps 6% 2.1s[===                           ] 11082049/bps 9% 1.6s[===                           ] 12736622/bps 12% 1.4s[====                          ] 13117959/bps 13% 1.3s[=====                         ] 14038046/bps 15% 1.2s[=====                  

-------------------------------------------------------------------------: 154.0-------------------------------------------------------------------------: 148.7>------------------------------------------------------------------------: 145.1>------------------------------------------------------------------------: 140.4>------------------------------------------------------------------------: 139.4>------------------------------------------------------------------------: 139.8>------------------------------------------------------------------------: 138.5>------------------------------------------------------------------------: 137.6=>-----------------------------------------------------------------------: 136.5=>-----------------------------------------------------------------------: 135.9=>-----------------------------------------------------------------------: 135.5=>-----------------------------------------------------------------------: 135.1=>--------------------------------------

==================================>---------------------------------------: 73.5===================================>--------------------------------------: 73.2===================================>--------------------------------------: 72.9===================================>--------------------------------------: 72.5===================================>--------------------------------------: 72.2===================================>--------------------------------------: 71.9====================================>-------------------------------------: 71.5====================================>-------------------------------------: 71.2====================================>-------------------------------------: 70.8====================================>-------------------------------------: 70.5====================================>-------------------------------------: 70.1====================================>-------------------------------------: 69.8=====================================>--

====================================================>---------------------: 41.2====================================================>---------------------: 40.9====================================================>---------------------: 40.6====================================================>---------------------: 40.2=====================================================>--------------------: 39.9=====================================================>--------------------: 39.5=====================================================>--------------------: 39.2=====================================================>--------------------: 38.8=====================================================>--------------------: 38.5======================================================>-------------------: 38.1======================================================>-------------------: 37.8======================================================>-------------------: 37.4========================================

======================================================================>----: 7.7======================================================================>----: 7.4=======================================================================>---: 7.0=======================================================================>---: 6.6=======================================================================>---: 6.2=======================================================================>---: 5.8=======================================================================>---: 5.5========================================================================>--: 5.1========================================================================>--: 4.7========================================================================>--: 4.3========================================================================>--: 3.9========================================================================>--: 3.5========================================

Training creates a model.json file and a weights.bin file which is utilized for prediction

## Prediction using REST API on the docker container

In [2]:
!s2i build . seldonio/seldon-core-s2i-nodejs:0.2-SNAPSHOT node-s2i-mnist-model:0.1

---> Installing application source...
---> Installing dependencies ...

> @tensorflow/tfjs-node@0.1.15 install /microservice/model/node_modules/@tensorflow/tfjs-node
> node scripts/install.js

* Downloading libtensorflow

* Building TensorFlow Node.js bindings

> protobufjs@6.8.8 postinstall /microservice/model/node_modules/protobufjs
> node scripts/postinstall

npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN nodejs_mnist@1.0.0 No repository field.
npm WARN nodejs_mnist@1.0.0 No license field.

added 50 packages from 57 contributors and audited 64 packages in 9.551s
found 0 vulnerabilities

Build completed successfully


In [3]:
!docker run --name "nodejs_mnist_predictor" -d --rm -p 5000:5000 node-s2i-mnist-model:0.1

d00ae5ad36ba6621eb289235102a9106d9aa029c645453351130c48ce818e5eb


Send some random features that conform to the contract

In [4]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p -t

----------------------------------------
SENDING NEW REQUEST:
RECEIVED RESPONSE:
Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 28
    shape: 28
    shape: 1
    values: 0.411
    values: 0.218
    values: 0.943
    values: 0.437
    values: 0.05
    values: 0.085
    values: 0.0
    values: 0.678
    values: 0.588
    values: 0.302
    values: 0.284
    values: 0.159
    values: 0.782
    values: 0.031
    values: 0.725
    values: 0.068
    values: 0.393
    values: 0.009
    values: 0.975
    values: 0.472
    values: 0.36
    values: 0.978
    values: 0.746
    values: 0.386
    values: 0.407
    values: 0.275
    values: 0.385
    values: 0.628
    values: 0.855
    values: 0.901
    values: 0.52
    values: 0.253
    values: 0.043
    values: 0.752
    values: 0.272
    values: 0.806
    values: 0.658
    values: 0.183
    values: 0.788
    values: 0.653
    values: 0.25
    values: 0.345
    values: 0.138
    values: 0.353
    values: 0.258
    values: 

In [5]:
!docker rm nodejs_mnist_predictor --force

nodejs_mnist_predictor


## Prediction using GRPC API on the docker container

In [6]:
!s2i build -E ./.s2i/environment_grpc . seldonio/seldon-core-s2i-nodejs:0.2-SNAPSHOT node-s2i-mnist-model:0.2

---> Installing application source...
---> Installing dependencies ...

> @tensorflow/tfjs-node@0.1.15 install /microservice/model/node_modules/@tensorflow/tfjs-node
> node scripts/install.js

* Downloading libtensorflow

* Building TensorFlow Node.js bindings

> protobufjs@6.8.8 postinstall /microservice/model/node_modules/protobufjs
> node scripts/postinstall

npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN nodejs_mnist@1.0.0 No repository field.
npm WARN nodejs_mnist@1.0.0 No license field.

added 50 packages from 57 contributors and audited 64 packages in 8.297s
found 0 vulnerabilities

Build completed successfully


In [7]:
!docker run --name "nodejs_mnist_predictor" -d --rm -p 5000:5000 node-s2i-mnist-model:0.2

7059dc3cd930b287c0eff2649034e338ddbaff4ce868c9fc695428bd426c04bd


Send some random features that conform to the contract

In [8]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p -t --grpc

----------------------------------------
SENDING NEW REQUEST:
RECEIVED RESPONSE:
Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 28
    shape: 28
    shape: 1
    values: 0.401
    values: 0.774
    values: 0.944
    values: 0.816
    values: 0.069
    values: 0.403
    values: 0.572
    values: 0.724
    values: 0.972
    values: 0.433
    values: 0.697
    values: 0.868
    values: 0.523
    values: 0.937
    values: 0.06
    values: 0.921
    values: 0.726
    values: 0.336
    values: 0.635
    values: 0.242
    values: 0.648
    values: 0.104
    values: 0.615
    values: 0.873
    values: 0.224
    values: 0.88
    values: 0.99
    values: 0.92
    values: 0.234
    values: 0.723
    values: 0.462
    values: 0.346
    values: 0.034
    values: 0.037
    values: 0.426
    values: 0.656
    values: 0.24
    values: 0.375
    values: 0.818
    values: 0.998
    values: 0.129
    values: 0.108
    values: 0.1
    values: 0.633
    values: 0.518
    values: 0

In [9]:
!docker rm nodejs_mnist_predictor --force

nodejs_mnist_predictor


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [10]:
!minikube start --memory 4096

😄  minikube v0.34.1 on linux (amd64)
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.13.3 ...
🚀  Launching Kubernetes v1.13.3 using kubeadm ... 
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


In [11]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [12]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [13]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


In [14]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

NAME:   seldon-core-crd
LAST DEPLOYED: Wed Mar 13 10:55:54 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     5s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  1s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          1s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        1s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  1s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  1s


NOTES:
NOTES: TODO


NAME:   seldon-core
LAST DEPLOYED: Wed Mar 13 10:55:59 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Pod(related)
NAME                                                 READY  

In [15]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-nodejs:0.2-SNAPSHOT node-s2i-mnist-model:0.1

---> Installing application source...
---> Installing dependencies ...

> @tensorflow/tfjs-node@0.1.15 install /microservice/model/node_modules/@tensorflow/tfjs-node
> node scripts/install.js

* Downloading libtensorflow

* Building TensorFlow Node.js bindings

> protobufjs@6.8.8 postinstall /microservice/model/node_modules/protobufjs
> node scripts/postinstall

npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN nodejs_mnist@1.0.0 No repository field.
npm WARN nodejs_mnist@1.0.0 No license field.

added 50 packages from 57 contributors and audited 64 packages in 10.376s
found 0 vulnerabilities

Build completed successfully


In [16]:
!kubectl create -f nodejs_mnist_deployment.json

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [18]:
!kubectl rollout status deploy/nodejs-mnist-deployment-nodejs-mnist-predictor-5aa9edd

deployment "nodejs-mnist-deployment-nodejs-mnist-predictor-5aa9edd" successfully rolled out


In [19]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

RECEIVED RESPONSE:
Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 28
    shape: 28
    shape: 1
    values: 0.73
    values: 0.582
    values: 0.585
    values: 0.096
    values: 0.041
    values: 0.568
    values: 0.079
    values: 0.363
    values: 0.454
    values: 0.336
    values: 0.864
    values: 0.376
    values: 0.068
    values: 0.937
    values: 0.502
    values: 0.639
    values: 0.635
    values: 0.386
    values: 0.599
    values: 0.559
    values: 0.892
    values: 0.825
    values: 0.577
    values: 0.39
    values: 0.184
    values: 0.293
    values: 0.366
    values: 0.082
    values: 0.083
    values: 0.894
    values: 0.213
    values: 0.975
    values: 0.052
    values: 0.055
    values: 0.999
    values: 0.824
    values: 0.337
    values: 0.534
    values: 0.482
    values: 0.445
    values: 0.199
    values: 0.537
    values: 0.193
    values: 0.922
    values: 0.874
    values: 0.07
    values: 0.315
    values: 0.992
    values: 0.384

In [20]:
!minikube delete

🔥  Deleting "minikube" from virtualbox ...
💔  The "minikube" cluster has been deleted.
